In [14]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64


# import CRUD Module
from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# set up connection to MongoDB
username = "aacuser"
password = "thepassword"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Add in Grazioso Salvare’s logo
image_filename = 'GSLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    # set up connection to MongoDB
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Nick Franklin Project 2'))),
    html.Center(html.P('Choose up to three animals to display their location on the map')),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        # Radio buttons for selecting filter options
        dcc.RadioItems(
            id = 'filter-type',
            # Grazioso Salvare required search keys
            options = [
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'MWR'},
                {'label': 'Disaster/Individual Tracking', 'value': 'DIT'},
                {'label': 'Reset - removes filters', 'value': 'RESET'}
            ],
            value = 'RESET',
            labelStyle = {'display': 'inline-block'}
        )
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
       #Features for the interactive data table to make it user-friendly for client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10, 
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    ### code to filter interactive data table with MongoDB queries
            # If desired search is Water Rescue
            if filter_type == 'WR':
                df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                 {'$or': [
                                                     {'breed': 'Labrador Retriever Mix'},
                                                     {'breed': 'Chesapeake Bay Retriever'},
                                                     {'breed': 'Newfoundland'},
                                                     {'breed': 'Newfoundland/Labrador Retriever'}]
                                                 },
                                                 {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                          {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                 }]})))
            # else if desired search is Mountain/Wilderness Rescue
            elif filter_type == 'MWR':
                df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                 {'$or': [
                                                     {'breed': 'German Shepherd'},
                                                     {'breed': 'Alaskan Malamute'},
                                                     {'breed': 'Old English Sheepdog'},
                                                     {'breed': 'Siberian Husky'},
                                                     {'breed': 'Rottweiler'}]
                                                 },
                                                 {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                          {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                 }]})))
            # else if desired search is Disaster/Individual Tracking
            elif filter_type == 'DIT':
                df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                 {'$or': [
                                                     {'breed': 'Doberman Pinscher'},
                                                     {'breed': 'German Shepherd'},
                                                     {'breed': 'Golden Retriever'},
                                                     {'breed': 'Bloodhound'},
                                                     {'breed': 'Rottweiler'}]
                                                 },
                                                 {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                          {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                 }]})))
            # else if desired search is reset
            elif filter_type == 'RESET':
                df = pd.DataFrame.from_records(shelter.read({}))

        
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
            data=df.to_dict('records')
        
        
            return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart)
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.histogram(dff, x = 'breed')
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', "selected_rows"),
     Input('datatable-id', "selected_columns")])
def update_map(viewData, selected_rows, selected_columns):
    # Code for the geolocation chart for selected animal
    dff=pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    #Only show the marker for the selected animal
    if selected_rows == []:
        selected_rows = [0]
    # map creation for only one animal selected
    if len(selected_rows) == 1:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Set up marker for animal on the map
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    # Set up the popup for each marker
                    dl.Popup([
                        html.H4("Name"),
                        html.P(dff.iloc[selected_rows[0],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0],15]),
                    ])
                ])
            ])
        ]
    # map creation for two animals selected
    elif len(selected_rows) == 2:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Set up marker for animal on the map
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    # Set up the popup for each marker
                    dl.Popup([
                        html.H4("Name"),
                        html.P(dff.iloc[selected_rows[0],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0],15]),
                    ])
                ]),
                # Set up marker for animal on the map
                dl.Marker(position=[(dff.iloc[selected_rows[1],13]), (dff.iloc[selected_rows[1],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[1],4]),
                    # Set up the popup for each marker
                    dl.Popup([
                        html.H4("Name"),
                        html.P(dff.iloc[selected_rows[1],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[1],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[1],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[1],15]),
                    ])
                ])
            ])
        ]
    # map creation for three animals selected
    elif len(selected_rows) == 3:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Set up marker for animal on the map
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    # Set up the popup for each marker
                    dl.Popup([
                        html.H4("Name"),
                        html.P(dff.iloc[selected_rows[0],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0],15]),
                    ])
                ]),
                # Set up marker for animal on the map
                dl.Marker(position=[(dff.iloc[selected_rows[1],13]), (dff.iloc[selected_rows[1],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[1],4]),
                    # Set up the popup for each marker
                    dl.Popup([
                        html.H4("Name"),
                        html.P(dff.iloc[selected_rows[1],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[1],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[1],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[1],15]),
                    ])
                ]),
                # Set up marker for animal on the map
                dl.Marker(position=[(dff.iloc[selected_rows[2],13]), (dff.iloc[selected_rows[2],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[2],4]),
                    # Set up the popup for each marker
                    dl.Popup([
                        html.H4("Name"),
                        html.P(dff.iloc[selected_rows[2],9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[2],12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[2],4]),
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[2],15]),
                    ])
                ])
            ])
        ]
app